## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('C:/Users/erich/UW_Boot_Camp/Analysis_Projects/World_Weather_Analysis/weather_database/WeatherPy_Database.csv')
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint George,US,37.1041,-113.5841,50.47,28,0,3.00,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,78.46,71,9,10.63,clear sky
2,2,Cockburn Town,TC,21.4612,-71.1419,75.67,69,0,15.84,clear sky
3,3,Airai,TL,-8.9266,125.4092,60.12,90,99,1.86,overcast clouds
4,4,Matay,EG,28.4190,30.7792,57.83,47,94,11.07,overcast clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,78.46,71,9,10.63,clear sky
2,2,Cockburn Town,TC,21.4612,-71.1419,75.67,69,0,15.84,clear sky
12,12,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,83,40,12.66,scattered clouds
22,22,Pitimbu,BR,-7.4706,-34.8086,86.59,61,21,11.34,few clouds
23,23,Atar,MR,20.5169,-13.0499,78.98,9,7,9.48,clear sky
25,25,Boende,CD,-0.2167,20.8667,77.85,44,46,2.33,scattered clouds
26,26,Atuona,PF,-9.8000,-139.0333,78.19,78,29,16.22,light rain
27,27,Cabo San Lucas,MX,22.8909,-109.9124,75.51,48,0,1.99,clear sky
31,31,Touros,BR,-5.1989,-35.4608,81.50,59,26,15.19,scattered clouds
32,32,Puerto Ayora,EC,-0.7393,-90.3518,82.36,87,83,5.99,broken clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

Unnamed: 0             181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,78.46,71,9,10.63,clear sky
2,2,Cockburn Town,TC,21.4612,-71.1419,75.67,69,0,15.84,clear sky
12,12,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,83,40,12.66,scattered clouds
22,22,Pitimbu,BR,-7.4706,-34.8086,86.59,61,21,11.34,few clouds
23,23,Atar,MR,20.5169,-13.0499,78.98,9,7,9.48,clear sky


In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,
2,Cockburn Town,TC,75.67,clear sky,21.4612,-71.1419,
12,Arraial Do Cabo,BR,84.16,scattered clouds,-22.9661,-42.0278,
22,Pitimbu,BR,86.59,few clouds,-7.4706,-34.8086,
23,Atar,MR,78.98,clear sky,20.5169,-13.0499,
25,Boende,CD,77.85,scattered clouds,-0.2167,20.8667,
26,Atuona,PF,78.19,light rain,-9.8000,-139.0333,
27,Cabo San Lucas,MX,75.51,clear sky,22.8909,-109.9124,
31,Touros,BR,81.50,scattered clouds,-5.1989,-35.4608,
32,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,People ThankYou
2,Cockburn Town,TC,75.67,clear sky,21.4612,-71.1419,Osprey Beach Hotel
12,Arraial Do Cabo,BR,84.16,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
22,Pitimbu,BR,86.59,few clouds,-7.4706,-34.8086,Reserva do Abiaí
23,Atar,MR,78.98,clear sky,20.5169,-13.0499,Oumou Elghoura
25,Boende,CD,77.85,scattered clouds,-0.2167,20.8667,
26,Atuona,PF,78.19,light rain,-9.8000,-139.0333,Villa Enata
27,Cabo San Lucas,MX,75.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
31,Touros,BR,81.50,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
32,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
nan_value = float("NaN") #Convert NaN values to empty string.
clean_hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,People ThankYou
2,Cockburn Town,TC,75.67,clear sky,21.4612,-71.1419,Osprey Beach Hotel
12,Arraial Do Cabo,BR,84.16,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
22,Pitimbu,BR,86.59,few clouds,-7.4706,-34.8086,Reserva do Abiaí
23,Atar,MR,78.98,clear sky,20.5169,-13.0499,Oumou Elghoura
26,Atuona,PF,78.19,light rain,-9.8000,-139.0333,Villa Enata
27,Cabo San Lucas,MX,75.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
31,Touros,BR,81.50,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
32,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
34,Mendoza,AR,81.93,scattered clouds,-32.8908,-68.8272,Hotel Mendoza


In [35]:
# 8a. Create the output File (CSV)
output_data_file='C:/Users/erich/UW_Boot_Camp/Analysis_Projects/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))